In [1]:
from langchain.document_loaders import SeleniumURLLoader

In [2]:
urls = [
    "https://thedecisionlab.com/reference-guide/neuroscience/behaviorism",
]

In [3]:
loader = SeleniumURLLoader(urls=urls)
data = loader.load()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/bharath/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [8]:
len(data)

1

In [10]:
data[0].metadata

{'source': 'https://thedecisionlab.com/reference-guide/neuroscience/behaviorism'}

In [9]:
from dotenv import load_dotenv
load_dotenv()


from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

In [16]:
docs = []
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
for doc in data:
	chunks = text_splitter.split_documents([doc])
	for chunkIndex, chunk in enumerate(chunks):
		docs.append(Document(page_content=chunk.page_content, metadata={
			"blockId": "l136hn5j24n",
			"chunkIndex": chunkIndex,
			"source": chunk.metadata["source"],
		}))

In [27]:
embeddings = OpenAIEmbeddings()

In [20]:
import os
from supabase.client import Client, create_client

supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_PRIVATE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

In [28]:
texts = []
metadatas = []
docEmbeddings = []
for doc in docs:
	texts.append(doc.page_content)
	metadatas.append(doc.metadata)
	e = embeddings.embed_documents([doc.page_content])
	docEmbeddings.append(e[0])

In [32]:
data, count = supabase.table('documents').insert({"content": texts[0], "metadata": metadatas[0], "embedding": docEmbeddings[0]}).execute()


In [33]:
for i in range(len(texts)):
	supabase.table('documents').insert({"content": texts[i], "metadata": metadatas[i], "embedding": docEmbeddings[i]}).execute()